**ALGORITMA YANG DIUSULKAN - BEST NASH PROBABILITAS**

In [7]:
pip install numpy
pip install time 
pip install concurrent 
pip install matplotlib
pip install scipy 
pip install json 
pip install python-docx
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import itertools
import random
import time
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
import json
from docx import Document
from tabulate import tabulate

class Robot:
    def __init__(self, id, position):
        self.id = id
        self.position = position
        self.neighbors = []
        self.preferences = []
        self.task_allocation = None
        self.utility = None
        self.social_utility = None

    def set_preferences(self, preferences):
        self.preferences = preferences

    def set_utility(self, utility):
        self.utility = utility

    def set_social_utility(self, social_utility):
        self.social_utility = social_utility

class Task:
    def __init__(self, id, position):
        self.id = id
        self.position = position

def createGraph(robots, adjacency_matrix):
    graph = {}
    robot_ids = list(robots.keys())

    for robot_id in robot_ids:
        position = robots[robot_id]
        robot = Robot(robot_id, position)
        graph[robot_id] = robot

    num_robots = len(robot_ids)
    for i in range(num_robots):
        robot = graph[robot_ids[i]]
        for j in range(num_robots):
            if adjacency_matrix[i, j] == 1:
                neighbor_id = robot_ids[j]
                neighbor_robot = graph[neighbor_id]
                robot.neighbors.append(neighbor_robot)

    return graph

def Max_distance(R, T):
    def euclidean_distance(point1, point2):
        return np.linalg.norm(np.array(point1) - np.array(point2))

    max_distances = {}
    for agent_name, agent_pos in R.items():
        max_distance = max(euclidean_distance(agent_pos, task_pos) for task_pos in T.values())
        max_distances[agent_name] = max_distance

    return max_distances

def calculateUtility(graph, tasks, R):
    utility_matrix = {}
    social_utility_matrix = {}
    task_objects = {task_id: Task(task_id, position) for task_id, position in tasks.items()}
    max_distances = Max_distance(R, tasks)
    distance_cache = {}

    for robot in graph.values():
        utilities = []
        social_utilities = []
        dmax = max_distances[robot.id]
        for task_id, task in task_objects.items():
            if (robot.id, task_id) not in distance_cache:
                distance_cache[(robot.id, task_id)] = calculate_distance(robot.position, task.position)
            distance = distance_cache[(robot.id, task_id)]
            # New utility calculation rule
            utility = (dmax - distance + 1)
            utilities.append((task_id, utility))
            social_utilities.append((task_id, utility))
        utility_matrix[robot.id] = utilities
        social_utility_matrix[robot.id] = social_utilities
        robot.set_utility(utilities)
        robot.set_social_utility(social_utilities)

    return utility_matrix, social_utility_matrix

def calculate_distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos2[1] - pos1[1]) ** 2)

def robotAwareness(graph):
    for robot in graph.values():
        sorted_utilities = sorted(robot.utility, key=lambda x: -x[1])
        robot.set_preferences([task_id for task_id, _ in sorted_utilities])

def get_neighbors(adjacency_matrix):
    neighbors = {}
    for i in range(adjacency_matrix.shape[0]):
        robot_id = f'R{i+1}'
        neighbor_indices = [j for j, val in enumerate(adjacency_matrix[i]) if val == 1]
        neighbor_ids = [f'R{j+1}' for j in neighbor_indices]
        neighbors[robot_id] = neighbor_ids
    return neighbors

def taskConflict(robot_id, preferences, neighbors):
    neighbor_ids = neighbors[robot_id]
    robot_preferences = preferences[robot_id]
    conflicts = []
    for neighbor_id in neighbor_ids:
        neighbor_preferences = preferences[neighbor_id]
        for i in range(len(robot_preferences)):
            if robot_preferences[i] == neighbor_preferences[i]:
                task = robot_preferences[i]
                conflicts.append(task)
                break
    return conflicts

def searchBestNashEquilibrium(preferences, utility_matrix, robot_ids, allocated_tasks):
    best_equilibrium = None
    best_score = -np.inf
    task_ids = list(set(itertools.chain(*preferences.values())))
    
    for perm in itertools.permutations(task_ids, len(robot_ids)):
        allocation = {robot_ids[i]: perm[i] for i in range(len(robot_ids))}
        sum_utilities = 0
        probabilities = []
        weighted_utilities = []

        for robot_id, task_id in allocation.items():
            utility = next(u for t, u in utility_matrix[robot_id] if t == task_id)
            weight = 1.5 if task_id not in allocated_tasks else 1.0
            weighted_utility = utility * weight
            weighted_utilities.append(weighted_utility)
            sum_utilities += weighted_utility

        total_weighted_utilities = sum(weighted_utilities)
        probabilities = [wu / total_weighted_utilities for wu in weighted_utilities]

        # Determine the Nash product of the probabilities
        nash_product = np.prod(probabilities)

        if nash_product > best_score:
            best_score = nash_product
            best_equilibrium = allocation

    return best_equilibrium

def Competitive(robots, tasks, adjacency_matrix):
    graph = createGraph(robots, adjacency_matrix)
    neighbors = get_neighbors(adjacency_matrix)
    task_allocation = {}
    allocated_tasks = set()

    utility_matrix, _ = calculateUtility(graph, tasks, robots)
    robotAwareness(graph)
    preferences = {robot.id: robot.preferences for robot in graph.values()}

    for robot_id in preferences.keys():
        conflicts = taskConflict(robot_id, preferences, neighbors)
        if conflicts:
            conflict_robot_ids = [robot_id] + neighbors[robot_id]
            best_nash_allocation = searchBestNashEquilibrium(preferences, utility_matrix, conflict_robot_ids, allocated_tasks)
            task_allocation.update(best_nash_allocation)
            allocated_tasks.update(best_nash_allocation.values())
        else:
            task_allocation[robot_id] = preferences[robot_id][0]
            allocated_tasks.add(preferences[robot_id][0])

    utility_matrix, social_utility_matrix = calculateUtility(graph, tasks, robots)
    
    # Adjust utilities for neighbors assigned to the same task
    for robot_id, robot in graph.items():
        for neighbor in robot.neighbors:
            if task_allocation[robot_id] == task_allocation[neighbor.id]:
                task_id = task_allocation[robot_id]
                utility_matrix[robot_id] = [(t, 0 if t == task_id else u) for t, u in utility_matrix[robot_id]]
                social_utility_matrix[robot_id] = [(t, 0 if t == task_id else su) for t, su in social_utility_matrix[robot_id]]

    return task_allocation, utility_matrix, social_utility_matrix


def sum_social_utility(social_utility_matrix):
    total_social_utility = sum(social_utility for utilities in social_utility_matrix.values() for _, social_utility in utilities)
    return total_social_utility

def calculate_resource_efficiency(utility_matrix, task_allocation):
    total_actual_utility = 0
    total_max_possible_utility = 0

    for robot_id, utilities in utility_matrix.items():
        max_possible_utility = max(utility for _, utility in utilities)
        task_id = task_allocation[robot_id]
        actual_utility = next(utility for task, utility in utilities if task == task_id)
        total_actual_utility += actual_utility
        total_max_possible_utility += max_possible_utility

    resource_efficiency = total_actual_utility * 100 / total_max_possible_utility if total_max_possible_utility > 0 else 0
    return resource_efficiency


R = {
    'R1': [41.73, 48.93],
    'R2': [4.97, 33.77],
    'R3': [90.27, 90.01],
    'R4': [94.48, 36.92],
    'R5': [49.09, 11.12]
}

T = {
    'T1': [78, 13],
    'T2': [39, 94],
    'T3': [24, 96],
    'T4': [40, 57],
    'T5': [10, 5]
}

adjacency_matrix = np.array([
    [0, 1, 0, 0, 1],
    [1, 0, 0, 0, 1],
    [0, 0, 0, 1, 0],
    [0, 0, 1, 0, 1],
    [1, 1, 0, 1, 0]
])

task_allocation, utility_matrix, social_utility_matrix = Competitive(R, T, adjacency_matrix)

# Print utility values for neighboring agents in the desired format
graph = createGraph(R, adjacency_matrix)
for robot_id, robot in graph.items():
    neighbor_ids = [neighbor.id for neighbor in robot.neighbors]
    print(f"\nChecking for Robot {robot_id} and its neighbors: {neighbor_ids}")
    print("Task Utility and Social Utility values:")
    for check_robot_id in [robot_id] + neighbor_ids:
        print(f"  Robot {check_robot_id}:")
        for task_id in T.keys():
            utility = next(u for t, u in utility_matrix[check_robot_id] if t == task_id)
            social_utility = next(su for t, su in social_utility_matrix[check_robot_id] if t == task_id)
            print(f"    Task {task_id} - Utility: {utility}, Social Utility: {social_utility}")

# Example usage of the calculateDistanceMatrix function with agent positions data
max_distances = Max_distance(R, T)
print("\nMaximum distances from each agent to any task:")
print(max_distances)

# Calculate and print resource efficiency
resource_efficiency = calculate_resource_efficiency(utility_matrix, task_allocation)
print("\nResource Efficiency:", resource_efficiency)

sum_soc_util = sum_social_utility(social_utility_matrix)
print("\nSocial Utility:", sum_soc_util)

# Print task allocation
print("\nTask allocation:")
for robot_id, task_id in task_allocation.items():
    print(f"{robot_id} -> {task_id}")



Checking for Robot R1 and its neighbors: ['R2', 'R5']
Task Utility and Social Utility values:
  Robot R1:
    Task T1 - Utility: 4.137078148172343, Social Utility: 4.137078148172343
    Task T2 - Utility: 10.038148140483344, Social Utility: 10.038148140483344
    Task T3 - Utility: 4.892266769095357, Social Utility: 4.892266769095357
    Task T4 - Utility: 46.9374029892156, Social Utility: 46.9374029892156
    Task T5 - Utility: 1.0, Social Utility: 1.0
  Robot R2:
    Task T1 - Utility: 1.0, Social Utility: 1.0
    Task T2 - Utility: 7.74739834687594, Social Utility: 7.74739834687594
    Task T3 - Utility: 11.851428860093975, Social Utility: 11.851428860093975
    Task T4 - Utility: 34.89357486090934, Social Utility: 34.89357486090934
    Task T5 - Utility: 47.71970751576596, Social Utility: 47.71970751576596
  Robot R5:
    Task T1 - Utility: 60.53951531453236, Social Utility: 60.53951531453236
    Task T2 - Utility: 6.018647715992429, Social Utility: 6.018647715992429
    Task T3 -

In [5]:
def load_positions_and_adjacency(doc_path):
    doc = Document(doc_path)
    positions_list = []
    adjacency_matrices_list = []
    
    sim_positions = []
    sim_adjacency = []
    
    for para in doc.paragraphs:
        if para.text.startswith('R'):
            _, pos = para.text.split(': ')
            sim_positions.append(list(map(float, pos.strip('[]').split(','))))
        elif para.text[0].isdigit():
            sim_adjacency.append(list(map(int, para.text.split())))
        elif para.text.startswith('Simulation') and sim_positions and sim_adjacency:
            positions_list.append(sim_positions)
            adjacency_matrices_list.append(np.array(sim_adjacency))
            sim_positions = []
            sim_adjacency = []
    
    if sim_positions and sim_adjacency:
        positions_list.append(sim_positions)
        adjacency_matrices_list.append(np.array(sim_adjacency))
    
    return positions_list, adjacency_matrices_list

# Load the saved positions and adjacency matrices
positions_list, adjacency_matrices_list = load_positions_and_adjacency('Input Skenario Test1.docx')

# Define the task positions (fixed)
task_positions = [
    [78, 13],
    [39, 94],
    [24, 76],
    [40, 57],
    [10, 5],
    [51, 24]
    # [68, 38],
    # [41, 48],
    # [28, 63],
    # [14, 30]
]

def run_simulation_fixed_tasks(positions_list, adjacency_matrices_list, task_positions):
    success_rates = []
    sum_social_utilities = []
    resource_efficiencies = []

    for positions, adjacency_matrix in zip(positions_list, adjacency_matrices_list):
        agent_dict = {f'R{i+1}': positions[i] for i in range(len(positions))}
        task_dict = {f'T{i+1}': task_positions[i] for i in range(len(task_positions))}

        task_allocation, utility_matrix, social_utility_matrix = Competitive(agent_dict, task_dict, adjacency_matrix)
        unique_tasks_allocated = len(set(task_allocation.values()))
        success_rate = unique_tasks_allocated / len(task_positions) * 100

        sum_social_utility_value = sum_social_utility(social_utility_matrix)
        resource_efficiency_value = calculate_resource_efficiency(utility_matrix, task_allocation)

        success_rates.append(success_rate)
        sum_social_utilities.append(sum_social_utility_value)
        resource_efficiencies.append(resource_efficiency_value)

    return success_rates, sum_social_utilities, resource_efficiencies

# Run the simulations with the loaded data
success_rates, sum_social_utilities, resource_efficiencies = run_simulation_fixed_tasks(positions_list, adjacency_matrices_list, task_positions)

# Create a new Word document for the results
doc = Document()
doc.add_heading('Hasil Simulasi Skenario 1', 0)

# Add table for simulation data
table = doc.add_table(rows=1, cols=6)
table.style = 'Table Grid'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Simulation Number'
hdr_cells[1].text = 'Agent Positions'
hdr_cells[2].text = 'Adjacency Matrix'
hdr_cells[3].text = 'Success Rate (%)'
hdr_cells[4].text = 'Sum of Social Utility'
hdr_cells[5].text = 'Resource Efficiency'

# Populate the table with data
for i, (positions, adjacency_matrix, rate, sum_social_utility_value, resource_efficiency_value) in enumerate(zip(positions_list, adjacency_matrices_list, success_rates, sum_social_utilities, resource_efficiencies)):
    row_cells = table.add_row().cells
    row_cells[0].text = str(i + 1)
    positions_text = '\n'.join([f'R{j+1}: {pos}' for j, pos in enumerate(positions)])
    row_cells[1].text = positions_text
    adjacency_text = '\n'.join([' '.join(map(str, row)) for row in adjacency_matrix])
    row_cells[2].text = adjacency_text
    row_cells[3].text = f"{rate:.2f}"
    row_cells[4].text = f"{sum_social_utility_value:.2f}"
    row_cells[5].text = f"{resource_efficiency_value:.2f}"

# Save the document
doc.save('Hasil Simulasi 2-1.docx')

In [7]:
# Fixed agent positions
agent_positions = {
    'R1': [41.73, 48.93],
    'R2': [54.97, 43.77],
    'R3': [90.27, 90.01],
    'R4': [94.48, 36.92],
    'R5': [49.09, 11.12],
    'R6': [50.54, 71.27]
}

# Fixed adjacency matrix for 6 agents
agent_matrix = np.array([
    [0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [1, 0, 0, 0, 0, 0]
])

def load_task_positions(doc_path):
    doc = Document(doc_path)
    task_positions_list = []
    task_positions = {}

    for para in doc.paragraphs:
        if para.text.startswith('Simulation'):
            if task_positions:
                task_positions_list.append(task_positions)
                task_positions = {}
        elif para.text.startswith('T'):
            task_id, pos = para.text.split(': ')
            task_positions[task_id] = list(map(float, pos.strip('[]').split(',')))
    
    if task_positions:
        task_positions_list.append(task_positions)
    
    return task_positions_list

# Define the functions needed for simulation
def run_simulation(agent_positions, task_positions, agent_matrix):
    task_allocation, utility_matrix, social_utility_matrix = Competitive(agent_positions, task_positions, agent_matrix)
    unique_tasks_allocated = len(set(task_allocation.values()))
    success_rate = unique_tasks_allocated / len(task_positions) * 100
    sum_soc_util = sum_social_utility(social_utility_matrix)
    resource_efficiency = calculate_resource_efficiency(utility_matrix, task_allocation)
    return success_rate, sum_soc_util, resource_efficiency

# Load the task positions from the document
task_positions_list = load_task_positions('Input Skenario Test2.docx')

results = []

for i, task_positions in enumerate(task_positions_list):
    success_rate, sum_soc_util, resource_efficiency = run_simulation(agent_positions, task_positions, agent_matrix)
    results.append((i + 1, task_positions, success_rate, sum_soc_util, resource_efficiency))
    
def save_results_to_doc(results, doc_path):
    doc = Document()
    doc.add_heading('Hasil Simulasi Skenario 2', level=1)
    table = doc.add_table(rows=1, cols=5)
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Simulation Number'
    hdr_cells[1].text = 'Task Positions'
    hdr_cells[2].text = 'Success Rate (%)'
    hdr_cells[3].text = 'Sum of Social Utility'
    hdr_cells[4].text = 'Resource Efficiency'

    for sim_num, task_positions, success_rate, sum_soc_util, resource_efficiency in results:
        row_cells = table.add_row().cells
        row_cells[0].text = str(sim_num)
        row_cells[1].text = '\n'.join([f'{task_id}: {pos}' for task_id, pos in task_positions.items()])
        row_cells[2].text = f"{success_rate:.2f}"
        row_cells[3].text = f"{sum_soc_util:.2f}"
        row_cells[4].text = f"{resource_efficiency:.2f}"

    doc.save(doc_path)

# Save the results to a Word document
save_results_to_doc(results, 'Hasil Simulasi 2-2.docx')



**TIME RESULT - RANDOM AGENT WITH SAME AMOUNT OF RANDOM TASK**

In [8]:
import time
from docx import Document
import numpy as np
import random

# Function to format agent positions and adjacency matrix
def format_agents_positions(R):
    formatted_positions = "\n".join([f"{agent}: [{pos[0]:.1f}, {pos[1]:.1f}]" for agent, pos in R.items()])
    return formatted_positions

def format_adjacency_matrix(adj_matrix):
    formatted_matrix = "\n".join(" ".join(map(str, row)) for row in adj_matrix)
    return formatted_matrix

# Function to run the simulation and measure execution time
def run_simulation(R, T, adjacency_matrix):
    start_time = time.time()
    task_allocation, utility_matrix, social_utility_matrix = Competitive(R, T, adjacency_matrix)
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time, R, adjacency_matrix

# Function to run multiple simulations and record results
def run_multiple_simulations(num_agents, num_simulations=50):
    results = []
    total_time = 0

    for sim_num in range(num_simulations):
        R = {f'R{i+1}': [random.uniform(0, 100), random.uniform(0, 100)] for i in range(num_agents)}
        T = {f'T{i+1}': [random.uniform(0, 100), random.uniform(0, 100)] for i in range(num_agents)}
        adjacency_matrix = np.random.randint(0, 2, size=(num_agents, num_agents))
        np.fill_diagonal(adjacency_matrix, 0)  # Ensure no self-loops
        execution_time, agents_positions, adj_matrix = run_simulation(R, T, adjacency_matrix)
        results.append((execution_time, agents_positions, adj_matrix))
        total_time += execution_time

    average_time = total_time / num_simulations
    return results, total_time, average_time

# Function to save results to Word document
def save_results_to_word(scenario_results):
    document = Document()
    
    for num_agents, (results, total_time, average_time) in scenario_results.items():
        document.add_heading(f'Scenario with {num_agents} agents', level=1)
        
        table = document.add_table(rows=1, cols=5)
        table.style = 'Table Grid'
        
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'No'
        hdr_cells[1].text = 'Number of Agents'
        hdr_cells[2].text = 'Agent Positions'
        hdr_cells[3].text = 'Adjacency Matrix'
        hdr_cells[4].text = 'Execution Time (s)'
        
        for i, (exec_time, agents_positions, adj_matrix) in enumerate(results):
            row_cells = table.add_row().cells
            row_cells[0].text = str(i + 1)
            row_cells[1].text = str(num_agents)
            row_cells[2].text = format_agents_positions(agents_positions)
            row_cells[3].text = format_adjacency_matrix(adj_matrix)
            row_cells[4].text = f'{exec_time:.4f}'
        
        document.add_paragraph(f'Total Execution Time: {total_time:.4f} s')
        document.add_paragraph(f'Average Execution Time: {average_time:.4f} s')
        document.add_page_break()
    
    document.save('Position Agent Change With Random Task (Same Amount) - Time Result.docx')

# Run the simulations for different numbers of agents
scenario_results = {}
for num_agents in range(5, 11):
    results, total_time, average_time = run_multiple_simulations(num_agents)
    scenario_results[num_agents] = (results, total_time, average_time)

# Save the results to a Word document
save_results_to_word(scenario_results)


KeyboardInterrupt: 

**TIME RESULT - RANDOM AGENT WITH 5 FIXED TASK**

In [ ]:
# from docx import Document
# import random
# import numpy as np

# # Function to generate random agent positions
# def generate_random_agent_positions(num_agents):
#     return {f'R{i+1}': [random.uniform(0, 100), random.uniform(0, 100)] for i in range(num_agents)}

# # Function to generate random adjacency matrix
# def generate_random_adjacency_matrix(num_agents):
#     matrix = np.random.randint(0, 2, size=(num_agents, num_agents))
#     np.fill_diagonal(matrix, 0)  # No self-loops
#     matrix = np.tril(matrix) + np.tril(matrix, -1).T  # Symmetric matrix
#     return matrix

# # Function to format agent positions and adjacency matrix
# def format_agents_positions(R):
#     formatted_positions = "\n".join([f"{agent}: [{pos[0]:.1f}, {pos[1]:.1f}]" for agent, pos in R.items()])
#     return formatted_positions

# def format_adjacency_matrix(adj_matrix):
#     formatted_matrix = "\n".join(" ".join(map(str, row)) for row in adj_matrix)
#     return formatted_matrix

# # Fixed task positions
# task_positions = [
#     [78, 13],
#     [39, 94],
#     [24, 76],
#     [40, 57],
#     [10, 5]
# ]

# # Function to create the initial Word document with simulation parameters
# def create_initial_document(num_simulations, num_agents_list, task_positions, filename='Data Time Result of Random Agent.docx'):
#     document = Document()
#     document.add_heading('Simulation Parameters', level=1)
    
#     for num_agents in num_agents_list:
#         document.add_heading(f'Number of Agents: {num_agents}', level=2)
#         for sim_num in range(num_simulations):
#             R = generate_random_agent_positions(num_agents)
#             adjacency_matrix = generate_random_adjacency_matrix(num_agents)
#             T = {f'T{i+1}': pos for i, pos in enumerate(task_positions)}
            
#             table = document.add_table(rows=1, cols=5)
#             table.style = 'Table Grid'
            
#             hdr_cells = table.rows[0].cells
#             hdr_cells[0].text = 'No'
#             hdr_cells[1].text = 'Number of Agents'
#             hdr_cells[2].text = 'Agent Positions'
#             hdr_cells[3].text = 'Adjacency Matrix'
#             hdr_cells[4].text = 'Task Positions'
            
#             row_cells = table.add_row().cells
#             row_cells[0].text = str(sim_num + 1)
#             row_cells[1].text = str(num_agents)
#             row_cells[2].text = format_agents_positions(R)
#             row_cells[3].text = format_adjacency_matrix(adjacency_matrix)
#             row_cells[4].text = "\n".join([f"{task}: [{pos[0]}, {pos[1]}]" for task, pos in T.items()])
            
#             document.add_paragraph()
    
#     document.save(filename)

# # Example usage
# num_simulations = 50
# num_agents_list = range(5, 11)
# create_initial_document(num_simulations, num_agents_list, task_positions)


In [9]:
# Function to format agent positions and adjacency matrix
def format_agents_positions(R):
    formatted_positions = "\n".join([f"{agent}: [{pos[0]:.1f}, {pos[1]:.1f}]" for agent, pos in R.items()])
    return formatted_positions

def format_adjacency_matrix(adj_matrix):
    formatted_matrix = "\n".join(" ".join(map(str, row)) for row in adj_matrix)
    return formatted_matrix

# Function to run the simulation and measure execution time
def run_simulation(R, T, adjacency_matrix):
    start_time = time.time()
    task_allocation, utility_matrix, social_utility_matrix = Competitive(R, T, adjacency_matrix)
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time, R, adjacency_matrix

# Function to read simulation parameters from the initial document
def read_simulation_parameters(filename='Data Time Result of Random Agent.docx'):
    document = Document(filename)
    simulation_params = []
    
    current_num_agents = None
    for table in document.tables:
        num_agents = int(table.cell(1, 1).text)
        agent_positions = {}
        pos_lines = table.cell(1, 2).text.split('\n')
        
        for line in pos_lines:
            agent, pos = line.split(': ')
            pos = pos.strip('[]').split(', ')
            agent_positions[agent] = [float(pos[0]), float(pos[1])]
        
        adj_matrix = []
        adj_lines = table.cell(1, 3).text.split('\n')
        
        for line in adj_lines:
            adj_matrix.append(list(map(int, line.split())))
        
        task_positions = {}
        task_lines = table.cell(1, 4).text.split('\n')
        
        for line in task_lines:
            task, pos = line.split(': ')
            pos = pos.strip('[]').split(', ')
            task_positions[task] = [float(pos[0]), float(pos[1])]
        
        if current_num_agents != num_agents:
            current_num_agents = num_agents
            simulation_params.append((num_agents, []))
        
        simulation_params[-1][1].append((agent_positions, np.array(adj_matrix), task_positions))
    
    return simulation_params

# Function to run multiple simulations and record results
def run_multiple_simulations(simulation_params, num_simulations=50):
    results_dict = {}
    
    for num_agents, scenarios in simulation_params:
        total_time = 0
        results = []

        for R, adj_matrix, T in scenarios:
            execution_time, agents_positions, adj_matrix = run_simulation(R, T, adj_matrix)
            results.append((execution_time, agents_positions, adj_matrix))
            total_time += execution_time

        average_time = total_time / num_simulations
        results_dict[num_agents] = (results, total_time, average_time)
    
    return results_dict

# Function to save results to Word document
def save_results_to_word(scenario_results):
    document = Document()
    
    for num_agents, (results, total_time, average_time) in scenario_results.items():
        document.add_heading(f'Scenario with {num_agents} agents', level=1)
        
        table = document.add_table(rows=1, cols=5)
        table.style = 'Table Grid'
        
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'No'
        hdr_cells[1].text = 'Number of Agents'
        hdr_cells[2].text = 'Agent Positions'
        hdr_cells[3].text = 'Adjacency Matrix'
        hdr_cells[4].text = 'Execution Time (s)'
        
        for i, (exec_time, agents_positions, adj_matrix) in enumerate(results):
            row_cells = table.add_row().cells
            row_cells[0].text = str(i + 1)
            row_cells[1].text = str(num_agents)
            row_cells[2].text = format_agents_positions(agents_positions)
            row_cells[3].text = format_adjacency_matrix(adj_matrix)
            row_cells[4].text = f'{exec_time:.4f}'
        
        document.add_paragraph(f'Total Execution Time: {total_time:.4f} s')
        document.add_paragraph(f'Average Execution Time: {average_time:.4f} s')
        document.add_page_break()
    
    # Summary Table
    document.add_heading('Summary', level=1)
    table = document.add_table(rows=2, cols=len(scenario_results) + 1)
    table.style = 'Table Grid'
    
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Time Result'
    for idx, num_agents in enumerate(scenario_results.keys(), start=1):
        hdr_cells[idx].text = f'{num_agents} Agents'
    
    row_total_time = table.rows[1].cells
    row_total_time[0].text = 'Total Time'
    row_avg_time = table.add_row().cells
    row_avg_time[0].text = 'Average'
    
    for idx, (num_agents, (results, total_time, average_time)) in enumerate(scenario_results.items(), start=1):
        row_total_time[idx].text = f'{total_time:.4f}'
        row_avg_time[idx].text = f'{average_time:.4f}'

    document.save('PROPOSED ALGORITHM - Time Result - Random Agent Positions (Fixed Task).docx')

# Update Competitive function to handle NoneType Errors when nash cannot be found. 
# this new function will let the residual agent to find the task with highest utility
def Competitive(robots, tasks, adjacency_matrix):
    graph = createGraph(robots, adjacency_matrix)
    neighbors = get_neighbors(adjacency_matrix)
    task_allocation = {}
    allocated_tasks = set()

    utility_matrix, _ = calculateUtility(graph, tasks, robots)
    robotAwareness(graph)
    preferences = {robot.id: robot.preferences for robot in graph.values()}

    for robot_id in preferences.keys():
        conflicts = taskConflict(robot_id, preferences, neighbors)
        if conflicts:
            conflict_robot_ids = [robot_id] + neighbors[robot_id]
            best_nash_allocation = searchBestNashEquilibrium(preferences, utility_matrix, conflict_robot_ids, allocated_tasks)
            if best_nash_allocation:
                task_allocation.update(best_nash_allocation)
                allocated_tasks.update(best_nash_allocation.values())
            else:
                # Fallback strategy: assign highest utility task available
                task_allocation[robot_id] = max(preferences[robot_id], key=lambda task: next(u for t, u in utility_matrix[robot_id] if t == task))
                allocated_tasks.add(task_allocation[robot_id])
        else:
            task_allocation[robot_id] = preferences[robot_id][0]
            allocated_tasks.add(preferences[robot_id][0])

    utility_matrix, social_utility_matrix = calculateUtility(graph, tasks, robots)
    
    # Adjust utilities for neighbors assigned to the same task
    for robot_id, robot in graph.items():
        for neighbor in robot.neighbors:
            if task_allocation[robot_id] == task_allocation[neighbor.id]:
                task_id = task_allocation[robot_id]
                utility_matrix[robot_id] = [(t, 0 if t == task_id else u) for t, u in utility_matrix[robot_id]]
                social_utility_matrix[robot_id] = [(t, 0 if t == task_id else su) for t, su in social_utility_matrix[robot_id]]

    return task_allocation, utility_matrix, social_utility_matrix

# Example usage
simulation_params = read_simulation_parameters('Data Time Result of Random Agent.docx')
scenario_results = run_multiple_simulations(simulation_params, num_simulations=50)
save_results_to_word(scenario_results)


**TIME RESULT - RANDOM TASK WITH 5 FIXED AGENT AND ADJACENCY MATRIX**

In [1]:
# from docx import Document
# import random
# import time
# import numpy as np

# # Function to format task positions
# def format_task_positions(T):
#     return "\n".join([f"{task}: [{pos[0]:.1f}, {pos[1]:.1f}]" for task, pos in T.items()])

# # Function to create the initial Word document with simulation parameters
# def create_initial_document(num_simulations, num_tasks_list, filename='Data Time Result of Random Task.docx'):
#     document = Document()
#     document.add_heading('Simulation Parameters', level=1)
    
#     for num_tasks in num_tasks_list:
#         document.add_heading(f'Number of Tasks: {num_tasks}', level=2)
#         for sim_num in range(num_simulations):
#             T = {f'T{i+1}': [random.uniform(0, 100), random.uniform(0, 100)] for i in range(num_tasks)}
#             table = document.add_table(rows=1, cols=4)
#             table.style = 'Table Grid'
            
#             hdr_cells = table.rows[0].cells
#             hdr_cells[0].text = 'No'
#             hdr_cells[1].text = 'Number of Tasks'
#             hdr_cells[2].text = 'Task Positions'
#             hdr_cells[3].text = 'Time'
            
#             row_cells = table.add_row().cells
#             row_cells[0].text = str(sim_num + 1)
#             row_cells[1].text = str(num_tasks)
#             row_cells[2].text = format_task_positions(T)
#             row_cells[3].text = ''
            
#             document.add_paragraph()
    
#     document.save(filename)

# num_simulations = 50
# num_tasks_list = range(5, 11)
# create_initial_document(num_simulations, num_tasks_list)

In [11]:
# Function to format task positions
def format_task_positions(T):
    return "\n".join([f"{task}: [{pos[0]:.1f}, {pos[1]:.1f}]" for task, pos in T.items()])

# Function to read simulation parameters from the initial document
def read_simulation_parameters(filename='Data Time Result of Random Task.docx'):
    document = Document(filename)
    simulation_params = []
    
    current_num_tasks = None
    for table in document.tables:
        num_tasks = int(table.cell(1, 1).text)
        task_positions = {}
        task_lines = table.cell(1, 2).text.split('\n')
        
        for line in task_lines:
            task, pos = line.split(': ')
            pos = pos.strip('[]').split(', ')
            task_positions[task] = [float(pos[0]), float(pos[1])]
        
        if current_num_tasks != num_tasks:
            current_num_tasks = num_tasks
            simulation_params.append((num_tasks, []))
        
        simulation_params[-1][1].append(task_positions)
    
    return simulation_params

# Function to run the simulation and measure execution time
def run_simulation(R, T, adjacency_matrix):
    start_time = time.time()
    task_allocation, utility_matrix, social_utility_matrix = Competitive(R, T, adjacency_matrix)
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time, T

# Function to run multiple simulations and record results
def run_multiple_simulations(simulation_data, a_position, adjacency_matrix):
    scenario_results = {}
    total_execution_times = {num_tasks: 0 for num_tasks, _ in simulation_data}

    for num_tasks, task_list in simulation_data:
        results = []
        for T in task_list:
            execution_time, tasks_positions = run_simulation(a_position, T, adjacency_matrix)
            results.append((execution_time, tasks_positions))
            total_execution_times[num_tasks] += execution_time

        scenario_results[num_tasks] = results

    average_execution_times = {num_tasks: total_time / len(scenario_results[num_tasks]) 
                               for num_tasks, total_time in total_execution_times.items()}

    return scenario_results, total_execution_times, average_execution_times

# Function to save results to Word document
def save_results_to_word(scenario_results, total_execution_times, average_execution_times):
    document = Document()
    
    for num_tasks, results in scenario_results.items():
        document.add_heading(f'Scenario with {num_tasks} tasks', level=1)
        
        table = document.add_table(rows=1, cols=4)
        table.style = 'Table Grid'
        
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'No'
        hdr_cells[1].text = 'Number of Tasks'
        hdr_cells[2].text = 'Task Positions'
        hdr_cells[3].text = 'Execution Time (s)'
        
        for i, (exec_time, tasks_positions) in enumerate(results):
            row_cells = table.add_row().cells
            row_cells[0].text = str(i + 1)
            row_cells[1].text = str(num_tasks)
            row_cells[2].text = "\n".join([f"{task}: [{pos[0]:.1f}, {pos[1]:.1f}]" for task, pos in tasks_positions.items()])
            row_cells[3].text = f'{exec_time:.4f}'
        
        document.add_paragraph(f'Total Execution Time: {total_execution_times[num_tasks]:.4f} s')
        document.add_paragraph(f'Average Execution Time: {average_execution_times[num_tasks]:.4f} s')
        document.add_page_break()
    
    # Add summary table
    document.add_heading('Summary', level=1)
    summary_table = document.add_table(rows=3, cols=7)
    summary_table.style = 'Table Grid'

    # Header
    hdr_cells = summary_table.rows[0].cells
    hdr_cells[0].text = 'Time Result'
    hdr_cells[1].text = '5 Task'
    hdr_cells[2].text = '6 Task'
    hdr_cells[3].text = '7 Task'
    hdr_cells[4].text = '8 Task'
    hdr_cells[5].text = '9 Task'
    hdr_cells[6].text = '10 Task'

    # Average Time
    avg_time_cells = summary_table.rows[1].cells
    avg_time_cells[0].text = 'Average Time'
    for i, num_tasks in enumerate(range(5, 11)):
        avg_time_cells[i+1].text = f'{average_execution_times[num_tasks]:.4f}'

    # Total Time
    total_time_cells = summary_table.rows[2].cells
    total_time_cells[0].text = 'Total Time'
    for i, num_tasks in enumerate(range(5, 11)):
        total_time_cells[i+1].text = f'{total_execution_times[num_tasks]:.4f}'

    document.save('PROPOSED ALGORITHM - Time Result - Random Task Positions (Fixed Agents and Adjacency).docx')

# Posisi agen dan adjacency matrix tetap
a_matrix = np.array([
    [0, 1, 0, 0, 0],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1]
])

a_position = {
    'R1': [41.73, 48.93],
    'R2': [54.97, 43.77],
    'R3': [90.27, 90.01],
    'R4': [94.48, 36.92],
    'R5': [49.09, 11.12]
}

# Load the initial document
simulation_data = read_simulation_parameters()

# Run the simulations for different numbers of tasks
scenario_results, total_execution_times, average_execution_times = run_multiple_simulations(simulation_data, a_position, a_matrix)

# Save the results to a Word document
save_results_to_word(scenario_results, total_execution_times, average_execution_times)


In [21]:
import time
from docx import Document
import numpy as np
import random

# Function to format agent positions and adjacency matrix
def format_agents_positions(R):
    formatted_positions = "\n".join([f"{agent}: [{pos[0]:.1f}, {pos[1]:.1f}]" for agent, pos in R.items()])
    return formatted_positions

def format_adjacency_matrix(adj_matrix):
    formatted_matrix = "\n".join(" ".join(map(str, row)) for row in adj_matrix)
    return formatted_matrix

# Function to run the simulation and measure execution time
def run_simulation(R, T, adjacency_matrix):
    start_time = time.time()
    task_allocation, utility_matrix, social_utility_matrix = Competitive(R, T, adjacency_matrix)
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time, R, adjacency_matrix, T

# Function to run multiple simulations and record results
def run_multiple_simulations(a_position, adjacency_matrix, num_tasks_list, num_simulations=50):
    scenario_results = {}

    for num_tasks in num_tasks_list:
        results = []
        total_time = 0

        for sim_num in range(num_simulations):
            T = {f'T{i+1}': [random.uniform(0, 100), random.uniform(0, 100)] for i in range(num_tasks)}
            execution_time, agents_positions, adj_matrix, tasks_positions = run_simulation(a_position, T, adjacency_matrix)
            results.append((execution_time, agents_positions, adj_matrix, tasks_positions))
            total_time += execution_time

        average_time = total_time / num_simulations
        scenario_results[num_tasks] = (results, total_time, average_time)

    return scenario_results

# Function to save results to Word document
def save_results_to_word(scenario_results):
    document = Document()
    
    for num_tasks, (results, total_time, average_time) in scenario_results.items():
        document.add_heading(f'Scenario with {num_tasks} tasks', level=1)
        
        table = document.add_table(rows=1, cols=5)
        table.style = 'Table Grid'
        
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = 'No'
        hdr_cells[1].text = 'Number of Tasks'
        hdr_cells[2].text = 'Agent Positions'
        hdr_cells[3].text = 'Adjacency Matrix'
        hdr_cells[4].text = 'Execution Time (s)'
        
        for i, (exec_time, agents_positions, adj_matrix, tasks_positions) in enumerate(results):
            row_cells = table.add_row().cells
            row_cells[0].text = str(i + 1)
            row_cells[1].text = str(num_tasks)
            row_cells[2].text = format_agents_positions(agents_positions)
            row_cells[3].text = format_adjacency_matrix(adj_matrix)
            row_cells[4].text = f'{exec_time:.4f}'
        
        document.add_paragraph(f'Total Execution Time: {total_time:.4f} s')
        document.add_paragraph(f'Average Execution Time: {average_time:.4f} s')
        document.add_page_break()
    
    document.save('Random Task Positions (Fixed Agents and Adjacency) - Time Result.docx')

# Posisi agen dan adjacency matrix tetap
a_matrix = np.array([
    [0, 1, 0, 0, 0],
    [1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1]
])

a_position = {
    'R1': [41.73, 48.93],
    'R2': [54.97, 43.77],
    'R3': [90.27, 90.01],
    'R4': [94.48, 36.92],
    'R5': [49.09, 11.12]
}

# Run the simulations for different numbers of tasks
num_tasks_list = range(5, 11)
scenario_results = run_multiple_simulations(a_position, a_matrix, num_tasks_list)

# Save the results to a Word document
save_results_to_word(scenario_results)

In [24]:
# import numpy as np
# import random
# from docx import Document

# # Function to generate random positions
# def generate_random_positions(agent_num, x_range=(0, 100), y_range=(0, 100)):
#     positions = []
#     for _ in range(agent_num):
#         x = round(random.uniform(*x_range), 1)
#         y = round(random.uniform(*y_range), 1)
#         positions.append([x, y])
#     return positions

# # Function to generate random adjacency matrix
# def generate_random_adjacency_matrix(agent_num):
#     matrix = np.zeros((agent_num, agent_num), dtype=int)
#     for i in range(agent_num):
#         neighbors = random.sample(range(agent_num), random.randint(1, 3))
#         for neighbor in neighbors:
#             if neighbor != i:
#                 matrix[i, neighbor] = 1
#                 matrix[neighbor, i] = 1  # Ensure the matrix is symmetric
#     return matrix

# # Generate positions and adjacency matrices
# simulations = 50
# agent_num = 5
# positions_list = []
# adjacency_matrices_list = []

# for _ in range(simulations):
#     positions = generate_random_positions(agent_num)
#     adjacency_matrix = generate_random_adjacency_matrix(agent_num)
#     positions_list.append(positions)
#     adjacency_matrices_list.append(adjacency_matrix)

# # Save positions and adjacency matrices to a Word document
# doc = Document()
# doc.add_heading('Simulasi dengan Skenario 1 (Posisi Agen dan Ketetanggaan Berubah)', 0)

# for i, (positions, adjacency_matrix) in enumerate(zip(positions_list, adjacency_matrices_list)):
#     doc.add_heading(f'Simulation {i+1}', level=1)
#     doc.add_heading('Agent Positions', level=2)
#     for j, pos in enumerate(positions):
#         doc.add_paragraph(f'R{j+1}: {pos}')
    
#     doc.add_heading('Adjacency Matrix', level=2)
#     for row in adjacency_matrix:
#         doc.add_paragraph(' '.join(map(str, row)))

# doc.save('Input Skenario Test11.docx')

In [25]:
# import random
# from docx import Document

# # Fixed agent positions
# agent_positions = {
#     'R1': [41.73, 48.93],
#     'R2': [54.97, 43.77],
#     'R3': [90.27, 90.01],
#     'R4': [94.48, 36.92],
#     'R5': [49.09, 11.12],
# }

# # Fixed adjacency matrix for 6 agents
# agent_matrix = np.array([
#     [0, 1, 0, 0, 0], 
#     [1, 0, 0, 0, 0], 
#     [0, 0, 0, 1, 0], 
#     [0, 0, 1, 0, 0], 
#     [0, 0, 0, 1, 1] 

# ])

# # Generate 100 sets of random task positions with one decimal place
# num_simulations = 50
# num_tasks = 5
# task_positions_list = []

# for _ in range(num_simulations):
#     task_positions = {f'T{i+1}': [round(random.uniform(0, 100), 1), round(random.uniform(0, 100), 1)] for i in range(num_tasks)}
#     task_positions_list.append(task_positions)

# # Save the generated task positions to a Word document
# def save_task_positions(doc_path, task_positions_list):
#     doc = Document()
#     for i, task_positions in enumerate(task_positions_list):
#         doc.add_heading(f'Simulation {i + 1}', level=1)
#         for task_id, pos in task_positions.items():
#             doc.add_paragraph(f'{task_id}: {pos}')
#     doc.save(doc_path)

# # Save the task positions to 'Input Skenario Test2.docx'
# save_task_positions('Input Skenario Test22.docx', task_positions_list)


In [ ]:
# # Fixed agent positions
# R = {
#     'R1': [41.73, 48.93],
#     'R2': [54.97, 43.77],
#     'R3': [90.27, 90.01],
#     'R4': [94.48, 36.92],
#     'R5': [49.09, 11.12],
#     'R6': [50.54, 71.27],
#     'R7': [70.07, 33.07],
#     'R8': [39.00, 71.00],
#     'R9': [24.00, 20.00],
#     'R10': [90.00, 57.00],
# }

# # Fixed adjacency matrix
# adjacency_matrix = np.array([
#     [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
#     [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
#     [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
#     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#     [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
#     [1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#     [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
#     [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
# ])

In [ ]:
# import time
# from docx import Document
# import numpy as np
# import random

# # Function to format agent positions and adjacency matrix
# def format_agents_positions(R):
#     formatted_positions = "\n".join([f"{agent}: [{pos[0]:.1f}, {pos[1]:.1f}]" for agent, pos in R.items()])
#     return formatted_positions

# def format_adjacency_matrix(adj_matrix):
#     formatted_matrix = "\n".join(" ".join(map(str, row)) for row in adj_matrix)
#     return formatted_matrix

# # Function to run the simulation and measure execution time
# def run_simulation(R, T, adjacency_matrix):
#     start_time = time.time()
#     task_allocation, utility_matrix, social_utility_matrix = Competitive(R, T, adjacency_matrix)
#     end_time = time.time()
#     execution_time = end_time - start_time
#     return execution_time, R, adjacency_matrix

# # Function to run multiple simulations and record results
# def run_multiple_simulations(num_agents, num_simulations=50):
#     results = []
#     total_time = 0

#     # Fixed task positions
#     task_positions = [
#         [78, 13],
#         [39, 94],
#         [24, 76],
#         [40, 57],
#         [10, 5]
#     ]
#     T = {f'T{i+1}': pos for i, pos in enumerate(task_positions)}

#     for sim_num in range(num_simulations):
#         R = {f'R{i+1}': [random.uniform(0, 100), random.uniform(0, 100)] for i in range(num_agents)}
#         adjacency_matrix = np.random.randint(0, 2, size=(num_agents, num_agents))
#         np.fill_diagonal(adjacency_matrix, 0)  # Ensure no self-loops
#         execution_time, agents_positions, adj_matrix = run_simulation(R, T, adjacency_matrix)
#         results.append((execution_time, agents_positions, adj_matrix))
#         total_time += execution_time

#     average_time = total_time / num_simulations
#     return results, total_time, average_time

# # Function to save results to Word document with 5 column
# def save_results_to_word(scenario_results):
#     document = Document()
    
#     for num_agents, (results, total_time, average_time) in scenario_results.items():
#         document.add_heading(f'Scenario with {num_agents} agents', level=1)
        
#         table = document.add_table(rows=1, cols=5)
#         table.style = 'Table Grid'
        
#         hdr_cells = table.rows[0].cells
#         hdr_cells[0].text = 'No'
#         hdr_cells[1].text = 'Number of Agents'
#         hdr_cells[2].text = 'Agent Positions'
#         hdr_cells[3].text = 'Adjacency Matrix'
#         hdr_cells[4].text = 'Execution Time (s)'
        
#         for i, (exec_time, agents_positions, adj_matrix) in enumerate(results):
#             row_cells = table.add_row().cells
#             row_cells[0].text = str(i + 1)
#             row_cells[1].text = str(num_agents)
#             row_cells[2].text = format_agents_positions(agents_positions)
#             row_cells[3].text = format_adjacency_matrix(adj_matrix)
#             row_cells[4].text = f'{exec_time:.4f}'
        
#         document.add_paragraph(f'Total Execution Time: {total_time:.4f} s')
#         document.add_paragraph(f'Average Execution Time: {average_time:.4f} s')
#         document.add_page_break()
    
#     document.save('Amount of Agent Change With Same 5 Task - Time Result.docx')

# # Update Competitive function to handle NoneType Errors when nash cannot be found. 
# # this new function will let the residual agent to find the task with highest utility
# def Competitive(robots, tasks, adjacency_matrix):
#     graph = createGraph(robots, adjacency_matrix)
#     neighbors = get_neighbors(adjacency_matrix)
#     task_allocation = {}
#     allocated_tasks = set()

#     utility_matrix, _ = calculateUtility(graph, tasks, robots, adjacency_matrix, task_allocation)
#     robotAwareness(graph, utility_matrix)
#     preferences = {robot.id: robot.preferences for robot in graph.values()}

#     for robot_id in preferences.keys():
#         conflicts = taskConflict(robot_id, preferences, neighbors)
#         if conflicts:
#             conflict_robot_ids = [robot_id] + neighbors[robot_id]
#             best_nash_allocation = searchBestNashEquilibrium(preferences, utility_matrix, conflict_robot_ids, allocated_tasks)
#             if best_nash_allocation:
#                 task_allocation.update(best_nash_allocation)
#                 allocated_tasks.update(best_nash_allocation.values())
#             else:
#                 # Fallback strategy: assign highest utility task available
#                 task_allocation[robot_id] = max(preferences[robot_id], key=lambda task: next(u for t, u in utility_matrix[robot_id] if t == task))
#                 allocated_tasks.add(task_allocation[robot_id])
#         else:
#             task_allocation[robot_id] = preferences[robot_id][0]
#             allocated_tasks.add(preferences[robot_id][0])

#     utility_matrix, social_utility_matrix = calculateUtility(graph, tasks, robots, adjacency_matrix, task_allocation)
    
#     # Adjust utilities for neighbors assigned to the same task
#     for robot_id, robot in graph.items():
#         for neighbor in robot.neighbors:
#             if task_allocation[robot_id] == task_allocation[neighbor.id]:
#                 task_id = task_allocation[robot_id]
#                 utility_matrix[robot_id] = [(t, 0 if t == task_id else u) for t, u in utility_matrix[robot_id]]
#                 social_utility_matrix[robot_id] = [(t, 0 if t == task_id else su) for t, su in social_utility_matrix[robot_id]]

#     return task_allocation, utility_matrix, social_utility_matrix

# # Run the simulations for different numbers of agents
# scenario_results = {}
# for num_agents in range(5, 11):
#     results, total_time, average_time = run_multiple_simulations(num_agents)
#     scenario_results[num_agents] = (results, total_time, average_time)

# # Save the results to a Word document
# save_results_to_word(scenario_results)
